# Section 2
Create a column which tracks the number of non-refunded orders made by a customer in a rolling 90-day window.

## Load

In [1]:
import pandas as pd
from datetime import date

# load raw data
df = pd.read_csv(
    'ecom_data.csv',
    dtype={
        'SalesOrder': str,
        'SKU': str,
        'Description': str,
        'UnitPrice': float,
        'CustomerID': int,
        'Channel': str,
        'State': str,
        'Sales': float,
        'Quantity': int
    },
    converters={
        'InvoiceDay': date.fromisoformat
    })

# drop duplicates
df.drop_duplicates(inplace=True)

# remove rows with Quantity == 0
df = df[df.Quantity != 0].reset_index(drop=True)

## Explore Refunded Orders

In [2]:
df[df.Sales < 0].head()

,SalesOrder,SKU,Description,UnitPrice,CustomerID,Channel,State,InvoiceDay,Sales,Quantity
166,C573283,22776,SWEETHEART 3 TIER CAKE STAND,9.95,18030,Email,IL,2011-10-28,-19.90,-2
184,C538341,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,SEO,CA,2010-12-10,-18.75,-5
247,C569114,22832,BROCANTE SHELF WITH HOOKS,10.75,14911,Email,CA,2011-09-30,-43.00,-4
356,C537788,16202E,BLACK PHOTO ALBUM,5.55,15916,Store,TN,2010-12-08,-11.10,-2
411,C566280,22723,SET OF 6 HERB TINS SKETCHBOOK,3.95,12748,Store,CA,2011-09-11,-31.60,-8


In [3]:
# examine ALARM CLOCK BAKELIKE GREEN orders from cust 15514
df[(df.SKU == '22726') & (df.CustomerID == 15514)].sort_values(['InvoiceDay', 'Quantity'])

,SalesOrder,SKU,Description,UnitPrice,CustomerID,Channel,State,InvoiceDay,Sales,Quantity
66762,C538341,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,Email,CA,2010-12-10,-22.50,-6
184,C538341,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,SEO,CA,2010-12-10,-18.75,-5
102277,C538341,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,Organic Social,CA,2010-12-10,-3.75,-1
12930,538313,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,Organic Social,CA,2010-12-10,3.75,1
172753,538313,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,Store,CA,2010-12-10,7.50,2
94464,538313,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,SEO,CA,2010-12-10,18.75,5
217895,538327,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,Store,CA,2010-12-10,52.50,14
217291,538809,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,Store,CA,2010-12-14,60.00,16
44067,574667,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,Email,CA,2011-11-06,7.50,2
28997,574667,22726,ALARM CLOCK BAKELIKE GREEN,3.75,15514,Store,CA,2011-11-06,157.50,42


Looks like orders can be entirely or partially refunded via multiple channels. For example - on 2010-12-10 this customer bought 22 clocks via orders via multiple channels and orders, and they also returned 12 clocks on two channels.

However, we cannot tell which of the two orders they returned clocks from. Without the ability to link refunds to a specific `SalesOrder`, it is impossible to link refunds to their actual order.

This behavior is not uncommon. For example:

In [4]:
df[(df.SKU == '22784') & (df.CustomerID == 12352)].sort_values('InvoiceDay')

,SalesOrder,SKU,Description,UnitPrice,CustomerID,Channel,State,InvoiceDay,Sales,Quantity
93083,C547388,22784,LANTERN CREAM GAZEBO,4.95,12352,SEO,LA,2011-03-22,-49.5,-10
127230,547390,22784,LANTERN CREAM GAZEBO,4.95,12352,Store,LA,2011-03-22,29.7,6


This customer returned more Gazebos than they bought. They likely bought some Gazebos before the start of our dataset (which begins on 2010-12-1), but we cannot link a refund to an order that does not exist in our data. Also consider this example:

In [5]:
df[(df.SKU == '23298') & (df.CustomerID == 12409)].sort_values('InvoiceDay')

,SalesOrder,SKU,Description,UnitPrice,CustomerID,Channel,State,InvoiceDay,Sales,Quantity
188974,556415,23298,SPOTTY BUNTING,4.15,12409,Store,VA,2011-06-10,664.00,160
105680,C563224,23298,SPOTTY BUNTING,4.95,12409,Organic Social,VA,2011-08-14,-4.95,-1
225456,C567643,23298,SPOTTY BUNTING,4.15,12409,Organic Social,VA,2011-09-21,-4.15,-1
228728,567642,23298,SPOTTY BUNTING,4.95,12409,SEO,VA,2011-09-21,29.70,6
148180,567721,23298,SPOTTY BUNTING,4.15,12409,Organic Social,VA,2011-09-22,161.85,39


This customer bought 160 Buntings on 2011-06-10. They then recieved two partial refunds for that order on 2011-08-14 and 2011-09-21. If they returned another Bunting after 2011-09-22, it would be impossible to tell which order they were returning items from.

All this being said, we need to define what a "non-refunded" order is, and determine a proxy to identify refunded orders.

We'll classify a "non-refunded" order as an order with zero refunds. Partially refunded orders (e.g. orders where a cust bought multiple items and returned fewer than they bought) will count as refunds.

Since there's no way to accurately identify refunded orders, we'll use a simple proxy: for a given 90 day window,  we'll count the number of orders they made and subtract the number of refunds.

First, we'll get the unique `SalesOrder` values for each customer and add a `Sale` column. This will be an integer column with a 1 for a sale and a -1 for a refund. We'll then calculate a 90 day window sum for each `CustomerID` for each `InvoiceDay`. Note that this number will include the `SalesOrder` value it's attached to.

In [6]:
# get unique SalesOrder, CustomerID, InvoiceDay
df_rolling_sales = df[['SalesOrder', 'CustomerID', 'InvoiceDay']].copy()
df_rolling_sales.drop_duplicates(inplace=True)
df_rolling_sales.reset_index(inplace=True, drop=True)

# add Sale column
df_rolling_sales['Sale'] = df_rolling_sales.SalesOrder.apply(lambda x: -1 if x[0] == 'C' else 1)
df_rolling_sales.head()

# find rolling sum values for non-refunded orders
from datetime import timedelta

non_refunded = []
for idx, row in df_rolling_sales.iterrows():
    # get window
    window = [row.InvoiceDay - timedelta(days=i) for i in range(0, 90)]
    
    # get cust orders
    df_cust = df_rolling_sales[df_rolling_sales.CustomerID == row.CustomerID]
    
    # sum and add to sales
    window_sum = df_cust[df_cust.InvoiceDay.isin(window)].Sale.sum()
    non_refunded.append(window_sum)
    
# add NonRefunded column to df_rolling_sales
df_rolling_sales['NonRefunded'] = pd.Series(non_refunded)

# drop columns from df_rolling_sales before join
df_rolling_sales = df_rolling_sales.drop(['CustomerID', 'InvoiceDay', 'Sale'], axis=1)
df_rolling_sales.set_index('SalesOrder', inplace=True)

# join non_refunded with dataset
df = df.set_index('SalesOrder')\
    .join(df_rolling_sales, how='left')\
    .reset_index(drop=False)

In [7]:
# confirm NonRefunded was added
df.head()

,SalesOrder,SKU,Description,UnitPrice,CustomerID,Channel,State,InvoiceDay,Sales,Quantity,NonRefunded
0,536365,71053,WHITE METAL LANTERN,3.39,17850,Store,KY,2010-12-01,40.68,12,9
1,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,17850,SEO,KY,2010-12-01,33.00,12,9
2,536365,22752,SET 7 BABUSHKA NESTING BOXES,7.65,17850,Store,KY,2010-12-01,30.60,4,9
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,17850,SEO,KY,2010-12-01,30.51,9,9
4,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,17850,SEO,KY,2010-12-01,22.95,9,9


Looks like it worked! Again, this is an imperfect proxy. But it's the best I can do without sinking too much time into it. Let's save it and move on.

In [8]:
df.to_csv('ecom_data_with_nonrefunded_orders.csv', index=False)

## Summary

Given the data, it is impossible to link refunds to the `SalesOrder` value they were issed for. Since there's no way to accurately identify refunded orders, I use a simple approximation: I subtracted the number of refunds a customer made during a 90 day peroid from the number of orders they made in the same window.